In [224]:
import pandas as pd
import numpy as np
import warnings
import pickle

In [225]:
tmp_data_path = '../MA_data/data/tmp/'
data_path = '../MA_data/data/'

s_year = 1997-1
e_year = 2020-1

read_new_tnic = True
store_tnic_info = True

In [221]:
def load_tnic2(data_path, s_year, e_year):
    tnic3 =  pd.read_csv(data_path+'tnic2_data/tnic2_data.txt', delimiter = '\t' )
    tnic = tnic3[(tnic3['year'] <= e_year) & (tnic3['year'] >= s_year)]


    tnic['gvkey1'] = tnic['gvkey1'].astype(str)
    tnic['gvkey2'] = tnic['gvkey2'].astype(str)

    tnic = tnic.dropna(subset=['score'])
    def add_rank(df):
        df['yearly_rank'] = df.groupby(['year','gvkey1'])['score'].rank('dense', ascending=False)
        df['yearly_rank'] = df['yearly_rank'].astype(int)
        return df

    tnic = add_rank(tnic)
    return tnic
    

In [226]:
def load_tnic3(data_path, s_year, e_year):
    tnic3 =  pd.read_csv(data_path+'tnic3_data/tnic3_data.txt', delimiter = '\t' )
    tnic = tnic3[(tnic3['year'] <= e_year) & (tnic3['year'] >= s_year)]


    tnic['gvkey1'] = tnic['gvkey1'].astype(str)
    tnic['gvkey2'] = tnic['gvkey2'].astype(str)

    tnic = tnic.dropna(subset=['score'])
    def add_rank(df):
        df['yearly_rank'] = df.groupby(['year','gvkey1'])['score'].rank('dense', ascending=False)
        df['yearly_rank'] = df['yearly_rank'].astype(int)
        return df

    tnic = add_rank(tnic)
    return tnic
    

In [176]:
def get_gvkeylst_adj(tnic, year, top_peer=10, get_adj=False, weighted=False):

#     def add_rank(df):
#         df['rank'] = df.groupby('gvkey1')['score'].rank('dense', ascending=False)
#         df['rank'] = df['rank'].astype(int)
#         return df
    
     tnic_year = tnic[tnic.year == year]

#     tnic_year = add_rank(tnic_year)

    tnic_year_top = tnic_year[tnic_year['yearly_rank'] <= top_peer].reset_index(drop=True)
    
    
    gvkey_arr = np.unique(tnic_year_top.gvkey1.unique())
    gvkey_lst = list(gvkey_arr)
    N = len(gvkey_lst)
    
    gvkey_index_mapping = {value:index for index, value in enumerate(gvkey_lst)}
    index_gvkey_mapping = {index:value for index, value in enumerate(gvkey_lst)}
    #return tnic_year_top, gvkey_arr gvkey_index_mapping, index_gvkey_mapping
    if get_adj:
        adj = np.zeros((N,N))
        
        for _, row in tnic_year_top.iterrows():
            
            gvkey1 = row['gvkey1']
            gvkey2 = row['gvkey2']
            score = row['score']
            
            index1 = gvkey_index_mapping[gvkey1]
            index2 = gvkey_index_mapping[gvkey2]
#            print(index1, index2)
            
            
            
            if weighted:
                adj[index1, index2] = score
            else:
                adj[index1, index2] = 1
            
        
        return gvkey_lst, gvkey_index_mapping, index_gvkey_mapping,   adj    
    else:
        return gvkey_lst, gvkey_index_mapping, index_gvkey_mapping
    
    
    
    

# Decide 4 types of MA

- SS
- PP
- PS
- SP

if  P, then the influence of deal is minor for P.
if  S, then the influence of deal is major for S.

Recall Master_1:

| ACU ok | AUP  ok | TCU ok | TUP ok | mark as                                           |
|------------------|-------------------|------------------|-------------------|---------------------------------------------------|
| 1                | 1                 | 1                | 1                 | 1                                                 |
| 1                | 1                 | 1                | 0                 | 2                                                 |
| 1                | 1                 | 0                | 1                 | 3                                                 |
| 1                | 0                 | 1                | 1                 | 4                                                 |
| 1                | 0                 | 1                | 0                 | 5                                                 |
| 1                | 0                 | 0                | 1                 | 6                                                 |
| 0                | 1                 | 1                | 1                 | 7                                                 |
| 0                | 1                 | 1                | 0                 | 8                                                 |
| 0                | 1                 | 0                | 1                 | 9                                                 |
|                  |                   |                  |                   | all other combination is certainly unanalysiable |

$\begin{array}{cl}0 & 130066 \\ 2 & 32609 \\ 6 & 10990 \\ 5 & 9219 \\ 8 & 7693 \\ 3 & 5720 \\ 1 & 3136 \\ 9 & 2277 \\ 4 & 1691 \\ 7 & 374\end{array}$


For those deal has multiple "identify" choices (like condition 1, it could be identified by any of 4 conditions), we want to make sure those GVKEY could be found in TNIC.


**Only 1, 2, 3, 4 has multiple choices**


## create list of gvkey list of each year

### Load data

In [145]:
sdc_df = pd.read_pickle(tmp_data_path+f'/sdc_analysable_{s_year+1}_{e_year+1}.pickle')

In [146]:
sdc_df.YEAR.unique()

array([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020])

In [229]:
tnic3 = load_tnic3(data_path, s_year, e_year)

<ipython-input-226-7dd0d46ce645>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnic['gvkey1'] = tnic['gvkey1'].astype(str)
<ipython-input-226-7dd0d46ce645>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnic['gvkey2'] = tnic['gvkey2'].astype(str)


In [230]:
tnic3.to_pickle(tmp_data_path+f"/tnic3_{s_year}_{e_year}")

In [216]:

if read_new_tnic:
    tnic = load_tnic(data_path, s_year, e_year)

    gvkey_lsts = {}
    key_ind_maps = {}
    ind_key_maps = {}


    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for year in range(s_year, e_year+1, 1): # year are integers
            gvkey_lsts[year], key_ind_maps[year], ind_key_maps[year] = get_gvkeylst_adj(tnic, year)
else:
    print("read TNIC from pickle")
    tnic = pd.read_pickle(tmp_data_path+f"/tnic2_{s_year}_{e_year}")
    with open(tmp_data_path+f"/tnic_info_3_pairs_{s_year}_{e_year}", 'rb') as f:
        gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)


if store_tnic_info:
    tnic.to_pickle(tmp_data_path+f"/tnic2_{s_year}_{e_year}")
if store_tnic_info:
    with open(tmp_data_path+f"/tnic_info_3_pairs_{s_year}_{e_year}", "wb") as f:
        pickle.dump((gvkey_lsts, key_ind_maps , ind_key_maps), f)
    

<ipython-input-215-14a588fc1198>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnic['gvkey1'] = tnic['gvkey1'].astype(str)
<ipython-input-215-14a588fc1198>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnic['gvkey2'] = tnic['gvkey2'].astype(str)


In [186]:

for y in range(s_year, e_year+1):
    print("number of unique firms in TNIC in year", y, " is ", len(gvkey_lsts[y]))

number of unique firms in year 1996  is  7528
number of unique firms in year 1997  is  7502
number of unique firms in year 1998  is  7302
number of unique firms in year 1999  is  7092
number of unique firms in year 2000  is  6739
number of unique firms in year 2001  is  6155
number of unique firms in year 2002  is  5696
number of unique firms in year 2003  is  5319
number of unique firms in year 2004  is  5184
number of unique firms in year 2005  is  5078
number of unique firms in year 2006  is  5016
number of unique firms in year 2007  is  4937
number of unique firms in year 2008  is  4678
number of unique firms in year 2009  is  4441
number of unique firms in year 2010  is  4291
number of unique firms in year 2011  is  4168
number of unique firms in year 2012  is  4060
number of unique firms in year 2013  is  4127
number of unique firms in year 2014  is  4267
number of unique firms in year 2015  is  4240
number of unique firms in year 2016  is  4092
number of unique firms in year 201

priority(1st to last): 
+ SS
+ SP
+ PS
+ PP

## mark

In [187]:
def mark_ma_type(df):
    '''
    SDC should match with most-recent-previous-year-update TNIC and Compustat
    
    '''
    
    def helper(row):
        if row.GVKEY_STATUS == '1': # 4 Scenarios, there is priority
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TS_GVKEY):
                return 'SS'
            elif (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TP_GVKEY):
                return "SP"
            elif (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TS_GVKEY):
                return "PS"
            elif (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TP_GVKEY):
                return "PP"
            else:
                return "XX"
            
            
        elif row.GVKEY_STATUS == "2": # 2 Scenarios 
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TS_GVKEY):
                return "SS"
            elif (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TS_GVKEY):
                return "PS"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "3": # 2 Scenarios 
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TP_GVKEY):
                return "SP"
            elif (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TP_GVKEY):
                return "PP"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "4": # 2 Scenarios 
            if (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TS_GVKEY):
                return "PS"
            elif (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TP_GVKEY):
                return "PP"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "7": # 2 Scenarios 
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TS_GVKEY):
                return "SS"
            elif (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TP_GVKEY):
                return "SP"
            else:
                return "XX"
            
            
            
        elif row.GVKEY_STATUS == "5": # 1 scenarios
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TS_GVKEY):
                return "SS"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "6": # 1 scenarios
            if (row.AS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AP_GVKEY != row.TP_GVKEY):
                return "SP"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "8": # 1 scenarios
            if (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TS_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TS_GVKEY):
                return "PS"
            else:
                return "XX"
        elif row.GVKEY_STATUS == "9": # 1 scenarios
            if (row.AP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.TP_GVKEY in gvkey_lsts[row.YEAR-1]) & (row.AS_GVKEY != row.TP_GVKEY):
                return "PP"
            else:
                return "XX"
        
    df['MA_TYPE'] = df.apply(helper, axis=1)
        
    return df
        
        

In [199]:
sdc_df2.GVKEY_STATUS.value_counts()

3    5720
1    3136
9    2277
2    1762
6    1036
5     452
7     374
8     352
4     280
Name: GVKEY_STATUS, dtype: int64

In [188]:
sdc_df2 = mark_ma_type(sdc_df)

In [189]:
sdc_df2.MA_TYPE.value_counts()

XX    5728
SP    4365
SS    3637
PP    1482
PS     177
Name: MA_TYPE, dtype: int64

In [190]:
for y in range(s_year+1, e_year+2):
    print(y, sdc_df2[(sdc_df2.YEAR == y) & (sdc_df2.MA_TYPE == "XX")].MA_TYPE.value_counts()/ sdc_df2[(sdc_df2.YEAR == y)].shape[0])

1997 XX    0.316327
Name: MA_TYPE, dtype: float64
1998 XX    0.326207
Name: MA_TYPE, dtype: float64
1999 XX    0.347965
Name: MA_TYPE, dtype: float64
2000 XX    0.389483
Name: MA_TYPE, dtype: float64
2001 XX    0.358511
Name: MA_TYPE, dtype: float64
2002 XX    0.362205
Name: MA_TYPE, dtype: float64
2003 XX    0.380704
Name: MA_TYPE, dtype: float64
2004 XX    0.411765
Name: MA_TYPE, dtype: float64
2005 XX    0.390516
Name: MA_TYPE, dtype: float64
2006 XX    0.447214
Name: MA_TYPE, dtype: float64
2007 XX    0.416048
Name: MA_TYPE, dtype: float64
2008 XX    0.40568
Name: MA_TYPE, dtype: float64
2009 XX    0.408081
Name: MA_TYPE, dtype: float64
2010 XX    0.465483
Name: MA_TYPE, dtype: float64
2011 XX    0.448889
Name: MA_TYPE, dtype: float64
2012 XX    0.399594
Name: MA_TYPE, dtype: float64
2013 XX    0.342593
Name: MA_TYPE, dtype: float64
2014 XX    0.335526
Name: MA_TYPE, dtype: float64
2015 XX    0.330317
Name: MA_TYPE, dtype: float64
2016 XX    0.301837
Name: MA_TYPE, dtype: float64
2

## SDC+TNIC 

- only obtain `MA_TYPE` is not XX

adding AGVKEY, TGVKEY. According to 4 types of MA:

In [207]:
def add_agvkey_tgvkey(df):
    def helper1(row):
        if row.MA_TYPE == 'SP':
            return row.AS_GVKEY
        elif row.MA_TYPE == 'SS':
            return row.AS_GVKEY
        elif row.MA_TYPE == 'PP':
            return row.AP_GVKEY
        elif row.MA_TYPE == 'PS':
            return row.AP_GVKEY

    def helper2(row):
        if row.MA_TYPE == 'SP':
            return row.TP_GVKEY
        elif row.MA_TYPE == 'SS':
            return row.TS_GVKEY
        elif row.MA_TYPE == 'PP':
            return row.TP_GVKEY
        elif row.MA_TYPE == 'PS':
            return row.TS_GVKEY


    df['AGVKEY'] = df.apply(helper1, axis=1)
    df['TGVKEY'] = df.apply(helper2, axis=1)

    return df
            
    

In [208]:
sdc_df2_noxx = sdc_df2[sdc_df2.MA_TYPE != "XX"]

In [209]:
sdc_df3 = add_agvkey_tgvkey(sdc_df2_noxx)

<ipython-input-207-64c237116adb>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AGVKEY'] = df.apply(helper1, axis=1)
<ipython-input-207-64c237116adb>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TGVKEY'] = df.apply(helper2, axis=1)


In [213]:
sdc_tnic = sdc_df3.copy()

In [214]:
print("MA events that could be present in TNIC network shape:", sdc_tnic.shape)

sdc_tnic.to_pickle(tmp_data_path+f"/sdc_tnic_{s_year+1}_{e_year+1}") # year using sdc year not tnic year

MA events that could be present in TNIC network shape: (9661, 50)
